In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from proj1_helpers import *
from data_helpers import *
from cross_validation import *
from plots import *

In [3]:
#LOAD TRAINING DATA
DATA_TRAIN_PATH = 'data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [4]:
#CLEAN TRAINING DATA
bounds = [0.0, 0.5]
tx_clean, ind_remov = treat_undefined_values(bounds, tX)
ind_remov

[4, 5, 6, 12, 26, 27, 28]

In [5]:
#MODEL BUILDING
tx, mean, std = standardize(tx_clean,0)
y,tx = build_model_data(tx,y)
y = classify(y)
num_samples = len(y)
num_features = tx.shape[1]
y = y.reshape(num_samples,1)


In [6]:
num_samples, num_features
tx.shape

(250000, 24)

In [7]:
from compute_gradient import *
from cost import *
from implementations import *

In [8]:
#Initial weight


#Maximum iterations through the whole set
max_iter = 100

#0.001 nan
#0.0005 nan

#Step-size
gammas = np.logspace(-6,-1, 7)
lambdas = np.logspace (-5,0,7)


#Regularization factor
# lambda_ = 0.01 142000 loss 10 000 iters
#lambda_ = 0.001 141000 loss 10 000 iters
#lambda_ = 0.02
#loss_min = np.inf
tx.shape,y.shape,y

((250000, 24), (250000, 1), array([[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]]))

In [9]:
losses_tr = np.zeros((len(gammas),len(lambdas)))
losses_val = np.zeros((len(gammas),len(lambdas)))

k_fold = 2
seed = 1
loss_min = np.inf
initial_w = np.zeros((tx.shape[1],1))
accuracy_max = 0

#CHOOSE LAMBDA AND GAMMA
for ind_lambda, lambda_ in enumerate(lambdas):
    for ind_gamma ,gamma_ in enumerate (gammas):
        loss_training = 0
        loss_validation = 0
        training_accuracy = 0
        validation_accuracy = 0
        for i,k in enumerate(range(k_fold)):
            
            tx_train, y_train, tx_val, y_val = cross_validation(y, tx, k, k_fold, seed)
            w, loss_train_k = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iter, gamma_)
            loss_val_k = reg_logistic_loss (y_val,tx_val,w,lambda_)/len(y_val)
            #loss_train_k /= len(y_train)
            loss_training += loss_train_k
            loss_validation += loss_val_k
            training_accuracy += predict_accuracy(y_train,tx_train,w)
            validation_accuracy += predict_accuracy(y_val,tx_val,w)
        
        training_accuracy /= k_fold
        validation_accuracy /= k_fold
        loss_training /= k_fold
        loss_validation /= k_fold
        print(lambda_)
        print(gamma_)
        print(loss_training)
        print(loss_validation)
        print(training_accuracy)
        print(validation_accuracy)
        if (validation_accuracy > accuracy_max):
            weights_star = w
            lambda_star = lambda_
            gamma_star = gamma_
            accuracy_max = validation_accuracy
        
        losses_tr[ind_lambda][ind_gamma] = loss_training
        losses_val[ind_lambda][ind_gamma] = loss_validation



1e-05
1e-06
69.14327595307076
0.6910804089800408
65.7332
65.7332
1e-05
6.8129206905796085e-06
68.0835881664114
0.6780215673432596
65.7332
65.7372
1e-05
4.641588833612782e-05
64.14678774531926
0.6280915945536306
66.46000000000001
66.5352
1e-05
0.00031622776601683794
58.302667364255974
0.5531699542828886
69.2132
69.2524
1e-05
0.0021544346900318843
55.963344992491756
0.5269656884168457
70.26400000000001
70.2424
1e-05
0.014677992676220705
53.38394043067041
0.5336927204498164
70.8432
70.8312


/Users/Sofia/Desktop/ML_Project1/cost.py:20: RuntimeWarning: divide by zero encountered in log
  loss = y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))


1e-05
0.1
35.7115200492278
nan
69.648
69.7332
6.812920690579608e-05
1e-06
62.24547398848923
nan
70.92
70.9188
6.812920690579608e-05
6.8129206905796085e-06
61.9751696413869
nan
70.9728
70.9808
6.812920690579608e-05
4.641588833612782e-05
60.70578934293728
0.5924645726436886
71.27279999999999
71.188
6.812920690579608e-05
0.00031622776601683794
58.78091363045647
0.5354657886303247
71.7628
71.7624
6.812920690579608e-05
0.0021544346900318843
57.434190452153246
0.5181621205803667
71.3148
71.29400000000001
6.812920690579608e-05
0.014677992676220705
53.53536286725249
0.5324496942807575
70.96000000000001
70.94319999999999
6.812920690579608e-05
0.1
35.701975090252546
nan
69.61359999999999
69.7132
0.0004641588833612782
1e-06
62.48886171201791
nan
70.884
70.88560000000001
0.0004641588833612782
6.8129206905796085e-06
62.212702175708884
nan
70.9428
70.9388
0.0004641588833612782
4.641588833612782e-05
60.91815515422012
0.591642198955932
71.244
71.1676
0.0004641588833612782
0.00031622776601683794
58.942

In [ ]:
lambdas

In [37]:
accuracy_max

71.8224

In [ ]:
gammas

In [ ]:
losses_tr, losses_val

In [38]:
loss_min, lambda_star, gamma_star

(inf, 0.0004641588833612782, 0.014677992676220705)

In [39]:
#PREDICT TRAINING ACCURACY
training_accuracy = predict_accuracy(y,tx,weights_star)
training_accuracy

65.6512

In [198]:
#LOAD TEST SET
DATA_TEST_PATH = 'data/test.csv'
ytest, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [203]:
#CLEAN AND STANDARDIZE TEST SET
tX_test_clean = np.delete(tX_test, ind_remov, axis=1)
tX_test_clean.shape


(568238, 23)

In [206]:
tx_test_clean = remove_undefined_values (tX_test_clean)
tx_test_clean.shape

(568238, 23)

In [207]:
tx_test_clean, _, _ = standardize(tx_test_clean,0)


In [210]:
tx_test_augmented = build_poly_all_features(tx_test_clean,2)

(568238, 23)

In [213]:
tx_test_augmented.shape

(568238, 46)

In [214]:
#BUILD TEST MODEL
ytest, tx_test = build_model_data(tx_test_augmented,ytest)

In [215]:
#PREDICT LABELS
OUTPUT_PATH = 'data/submission_reg_logistic-degre2.csv'
y_pred = predict_labels(weights_star, tx_test,'logistic')
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)